In [1]:
import json
jlo = lambda s: json.load(open(s,'r'))
%pylab inline
%config InlineBackend.figure_format = 'retina'
from itertools import combinations
import os
from scipy import interpolate
import pandas as pd
import glob as gl
import collections as co
import gln_v8_parallel_py3 as gln

Populating the interactive namespace from numpy and matplotlib


In [ ]:
## load raw data ##
a = pd.read_csv("Neuprint_Neurons_52a133.csv", skip_blank_lines = 1, usecols=['bodyId:long','bL(L):boolean'])
traced = pd.read_csv("traced-neurons.csv", skip_blank_lines = 1)

# select neurons in BL(L) region #
bl = a[a['bL(L):boolean']==1]
neurons = set(bl['bodyId:long']).intersection(set(traced['bodyId']))
neurons = list(neurons)
json.dump(neurons, open('./neurons_in_BL_L_mapped.json','w'))

all_neurons = gl.glob('./skeletons/*.swc')
included = set(neurons).intersection(set([int(i.split('/')[2][:-4]) for i in all_neurons]))
not_included = list(set(neurons).difference(included))


# trace morphology of neurons

In [ ]:
BL_L = {}
for idx in neurons:
    BL_L[idx] = {'post':[], 'pre':[], 'coor':[]}
    with open('./skeletons/%d.swc'%idx,'r') as f:
        next(f)
        next(f)
        next(f)
        for line in f:
            b,_,x,y,z,_,c = line.strip().split(' ')
            BL_L[idx]['post'].append(int(b))
            BL_L[idx]['pre'].append(int(c))
            BL_L[idx]['coor'].append([float(x),float(y),float(z)])

In [ ]:
links = {}
for idx in neurons:
    links[idx] = {}
    tmp = array(BL_L[idx]['pre'])
    degrees = [len(where(tmp==i)[0])+1 for i in range(1,max(BL_L[idx]['post'])+1)]
    degrees = array(degrees)
    brachings = where(degrees>2)[0]
    leaves = where(tmp==-1)[0]
    ends = where(degrees==1)[0]
    
    for i in range(len(BL_L[idx]['coor'])):
        if BL_L[idx]['post'][i]-1 in leaves:
            links[idx][len(links[idx])] = {'end_points':[i], 'points':[BL_L[idx]['coor'][i]], 'nodes':[i]}
        elif BL_L[idx]['post'][i]-1 in ends:
            links[idx][len(links[idx])-1]['end_points'].append(i)
            links[idx][len(links[idx])-1]['points'].append(BL_L[idx]['coor'][i])
            links[idx][len(links[idx])-1]['nodes'].append(i)  
        elif BL_L[idx]['post'][i]-1 in brachings:
            if len(links[idx][len(links[idx])-1]['end_points']) == 1:
                links[idx][len(links[idx])-1]['end_points'].append(i)
                links[idx][len(links[idx])-1]['points'].append(BL_L[idx]['coor'][i])
                links[idx][len(links[idx])-1]['nodes'].append(i)   
            else:
                links[idx][len(links[idx])] = {'end_points':[i], 'points':[BL_L[idx]['coor'][i]], 'nodes':[i]}
        elif BL_L[idx]['pre'][i]-1 in brachings:
            links[idx][len(links[idx])] = {'end_points':[BL_L[idx]['pre'][i]-1], 'points':[BL_L[idx]['coor'][BL_L[idx]['pre'][i]-1],BL_L[idx]['coor'][i]], 'nodes':[BL_L[idx]['pre'][i]-1,i]}
        else:
            links[idx][len(links[idx])-1]['points'].append(BL_L[idx]['coor'][i])
            links[idx][len(links[idx])-1]['nodes'].append(i)
    break
    

# connect neurons together into a big network based on synaptic connections 

In [6]:
a = pd.read_csv("Neuprint_SynapseSet_to_Synapses_52a133.csv", skip_blank_lines = 1)


In [7]:
neuron_connect = array([[i.split('_')[0], i.split('_')[1]] for i in a.index.values])

In [8]:
neurons = [str(i) for i in neurons]

In [9]:
neurons = array(neurons)

In [10]:
## find all rows with two neurons both in BL_L ##
mask = in1d(neuron_connect[:,0], neurons)*in1d(neuron_connect[:,1], neurons)

In [16]:
synapses = {}
a_BL = a['070097075'][mask]
for i in range(len(a_BL)):
    synapses[a_BL[i]] = {'neurons':[a_BL.index.values[i].split('_')[0], a_BL.index.values[i].split('_')[1]]}

In [20]:
synapses = {str(i):synapses[i] for i in synapses}

In [21]:
json.dump(synapses, open('./synapseId_2_neurons_BL_L.json','w'))

In [23]:
b = pd.read_csv("Neuprint_Synapse_Connections_52a133.csv", skip_blank_lines = 1)


In [25]:
synapses_array = array([int(i) for i in synapses])

In [26]:
## find all rows with two synapses both in BL_L ##
mask1 = in1d(b[':START_ID(Syn-ID)'], synapses_array)*in1d(b[':END_ID(Syn-ID)'], synapses_array)

In [48]:
b_BL = b[mask1]
synapse_connect = {}
for i in where(mask1==1)[0]:
    synapse_connect[b_BL[':START_ID(Syn-ID)'][i]] = b_BL[':END_ID(Syn-ID)'][i]
    synapse_connect[b_BL[':END_ID(Syn-ID)'][i]] = b_BL[':START_ID(Syn-ID)'][i]

In [52]:
synapse_connect = {str(i):str(synapse_connect[i]) for i in synapse_connect}

In [53]:
json.dump(synapse_connect, open('./synapse_connections_BL_L.json','w'))

In [64]:
neuron_2_synapse = {i:{} for i in neurons}
for i in synapses:
    j,k = synapses[i]['neurons']
    if k in neuron_2_synapse[j]:
        neuron_2_synapse[j][k].append(i)
    else:
        neuron_2_synapse[j][k] = [i]
#     if j in neuron_2_synapse[k]:
#         neuron_2_synapse[k][j].append(i)
#     else:
#         neuron_2_synapse[k][j] = [i]

In [65]:
json.dump(neuron_2_synapse, open('./neuron_2_synapse_BL_L.json','w'))

In [76]:
with open('./Neuprint_Synapses_52a133.csv','r') as f:
    next(f)
    for line in f:
        tmp = line.strip().split(',')
#         break
        if int(tmp[0]) in synapses_array:
            synapses[tmp[0]]['position'] = [int(tmp[3].strip()[4:]), int(tmp[4].strip()[2:]), int(tmp[5].strip()[2:-2])]

In [83]:
json.dump(synapses, open('./synapseId_2_neurons_BL_L.json','w'))

In [100]:
for idx in neurons:
#     links[idx] = {}
    idx = int(idx)
    links[idx] = {}
    tmp = array(BL_L[idx]['pre'])
    degrees = [len(where(tmp==i)[0])+1 for i in range(1,max(BL_L[idx]['post'])+1)]
    degrees = array(degrees)
    brachings = where(degrees>2)[0]
    leaves = where(tmp==-1)[0]
    ends = where(degrees==1)[0]
    
    for i in range(len(BL_L[idx]['coor'])):
        if BL_L[idx]['post'][i]-1 in leaves:
            links[idx][len(links[idx])] = {'end_points':[i], 'points':[BL_L[idx]['coor'][i]], 'nodes':[i]}
        elif BL_L[idx]['pre'][i]-1 in brachings:
            links[idx][len(links[idx])] = {'end_points':[BL_L[idx]['pre'][i]-1], 'points':[BL_L[idx]['coor'][BL_L[idx]['pre'][i]-1],BL_L[idx]['coor'][i]], 'nodes':[BL_L[idx]['pre'][i]-1,i]}
        
        if BL_L[idx]['post'][i]-1 in ends:
            links[idx][len(links[idx])-1]['end_points'].append(i)
            links[idx][len(links[idx])-1]['points'].append(BL_L[idx]['coor'][i])
            links[idx][len(links[idx])-1]['nodes'].append(i)  
        elif BL_L[idx]['post'][i]-1 in brachings:
            if len(links[idx][len(links[idx])-1]['end_points']) == 1:
                links[idx][len(links[idx])-1]['end_points'].append(i)
                links[idx][len(links[idx])-1]['points'].append(BL_L[idx]['coor'][i])
                links[idx][len(links[idx])-1]['nodes'].append(i)   
            else:
                links[idx][len(links[idx])] = {'end_points':[i], 'points':[BL_L[idx]['coor'][i]], 'nodes':[i]}
        elif (BL_L[idx]['post'][i]-1 not in leaves) and (BL_L[idx]['pre'][i]-1 not in brachings):
            links[idx][len(links[idx])-1]['points'].append(BL_L[idx]['coor'][i])
            links[idx][len(links[idx])-1]['nodes'].append(i)

In [101]:
links = {str(i):links[i] for i in links}
json.dump(links, open('./links_BL_L_before_attach_synapses.json','w'))

In [ ]:
for idx in neurons:            
    ## attach synapses ##
    tmp = array(BL_L[idx]['pre'])
    degrees = array([len(where(tmp==i)[0])+1 for i in range(1,max(BL_L[idx]['post'])+1)])
    brachings = where(degrees>2)[0]
    leaves = where(tmp==-1)[0]
    ends = where(degrees==1)[0]
    for j in neuron_2_synapse[str(idx)]:
        for k in neuron_2_synapse[str(idx)][j]:
            if 'position' not in synapses[k]:
                l = synapse_connect[k]
                if 'position' not in synapses[l]:
                    continue
                else:
                    syn_coor = synapses[l]['position']
            else:
                syn_coor = synapses[k]['position']
            points = array(BL_L[idx]['coor'])
            dis = sqrt(sum((points - array(syn_coor))**2, axis=1))
            nod = where(dis==min(dis))[0][0]
            if nod in leaves or nod in ends:
                for t in links[idx]:
                    if nod in links[idx][t]['end_points']:
                        if nod == links[idx][t]['end_points'][0]:
                            links[idx][t]['points'][0] = syn_coor
                        else:
                            links[idx][t]['points'][-1] = syn_coor
                        break
            elif nod in brachings:
                for t in links[idx]:
                    if nod in links[idx][t]['end_points']:
                        if nod == links[idx][t]['end_points'][0]:
                            links[idx][t]['points'][0] = syn_coor
                        else:
                            links[idx][t]['points'][-1] = syn_coor
            else:
                for t in links[idx]:
                    if nod in links[idx][t]['nodes']:
                        nod_idx = links[idx][t]['nodes'].index(nod)
                        links[idx][t]['points'][nod_idx] = syn_coor
                        links[idx][len(links[idx])] = {'end_points':[links[idx][t]['end_points'][0], nod],
                                                      'points':links[idx][t]['points'][:nod_idx+1],
                                                      'nodes':links[idx][t]['points'][:nod_idx+1]}
                        links[idx][t] = {'end_points':[nod, links[idx][t]['end_points'][1]],
                                            'points':links[idx][t]['points'][nod_idx:],
                                            'nodes':links[idx][t]['points'][nod_idx:]}

                        break

In [111]:
BL_L = {str(i):BL_L[i] for i in BL_L}

In [136]:
links = jlo('./links_BL_L_before_attach_synapses.json')
synapse_connect = jlo('./synapse_connections_BL_L.json')
synapses = jlo('./synapseId_2_neurons_BL_L.json')

In [ ]:
syn_visited = []
neuron_synapse_nodes = {i:[] for i in neurons}
for s in synapses_array:
    s = str(s)
    s1 = synapse_connect[s]
    if s not in syn_visited and s1 not in syn_visited:
        syn_visited.append(s)
        syn_visited.append(s1)
        if 'position' not in synapses[s]:
#             s1 = synapse_connect[s]
#             syn_visited.append(s1)
            if 'position' not in synapses[s1]:            
#                 del synapses[s]
#                 del synapses[s1]
#                 del synapse_connect[s1]
#                 del synapse_connect[s]
                continue
            else:
                n1,n2 = synapses[s]['neurons']
                syn_coor = synapses[s1]['position']
        else:
#             s1 = synapse_connect[s]
#             syn_visited.append(s1)
            syn_coor = synapses[s]['position']
            n1,n2 = synapses[s]['neurons']
        ## edit neuron n1 ##
#         n1 = int(n1)
#         n2 = int(n2)
        tmp = array(BL_L[n1]['pre'])
        degrees = array([len(where(tmp==i)[0])+1 for i in range(1,max(BL_L[n1]['post'])+1)])
        brachings = where(degrees>2)[0]
        leaves = where(tmp==-1)[0]
        ends = where(degrees==1)[0]
        points = array(BL_L[n1]['coor'])
        dis = sqrt(sum((points - array(syn_coor))**2, axis=1))
        nod = where(dis==min(dis))[0][0]
        neuron_synapse_nodes[str(n1)].append(nod)
        if nod in leaves or nod in ends:
            for t in links[n1]:
                if nod in links[n1][t]['end_points']:
                    if nod == links[n1][t]['end_points'][0]:
                        links[n1][t]['points'][0] = syn_coor
                    else:
                        links[n1][t]['points'][-1] = syn_coor
                    break
        elif nod in brachings:
            for t in links[n1]:
                if nod in links[n1][t]['end_points']:
                    if nod == links[n1][t]['end_points'][0]:
                        links[n1][t]['points'][0] = syn_coor
                    else:
                        links[n1][t]['points'][-1] = syn_coor
        else:
            for t in links[n1]:
                if nod in links[n1][t]['nodes']:
                    nod_idx = links[n1][t]['nodes'].index(nod)
                    links[n1][t]['points'][nod_idx] = syn_coor
                    links[n1][len(links[n1])] = {'end_points':[links[n1][t]['end_points'][0], nod],
                                                  'points':links[n1][t]['points'][:nod_idx+1],
                                                  'nodes':links[n1][t]['nodes'][:nod_idx+1]}
                    links[n1][t] = {'end_points':[nod, links[n1][t]['end_points'][1]],
                                        'points':links[n1][t]['points'][nod_idx:],
                                        'nodes':links[n1][t]['nodes'][nod_idx:]}

                    break
        
        ## edit neuron n2 ##
        tmp = array(BL_L[n2]['pre'])
        degrees = array([len(where(tmp==i)[0])+1 for i in range(1,max(BL_L[n2]['post'])+1)])
        brachings = where(degrees>2)[0]
        leaves = where(tmp==-1)[0]
        ends = where(degrees==1)[0]
        points = array(BL_L[n2]['coor'])
        dis = sqrt(sum((points - array(syn_coor))**2, axis=1))
        nod = where(dis==min(dis))[0][0]
        neuron_synapse_nodes[str(n2)].append(nod)
        if nod in leaves or nod in ends:
            for t in links[n2]:
                if nod in links[n2][t]['end_points']:
                    if nod == links[n2][t]['end_points'][0]:
                        links[n2][t]['points'][0] = syn_coor
                    else:
                        links[n2][t]['points'][-1] = syn_coor
                    break
        elif nod in brachings:
            for t in links[n2]:
                if nod in links[n2][t]['end_points']:
                    if nod == links[n2][t]['end_points'][0]:
                        links[n2][t]['points'][0] = syn_coor
                    else:
                        links[n2][t]['points'][-1] = syn_coor
        else:
            for t in links[n2]:
                if nod in links[n2][t]['nodes']:
                    nod_idx = links[n2][t]['nodes'].index(nod)
                    links[n2][t]['points'][nod_idx] = syn_coor
                    links[n2][len(links[n2])] = {'end_points':[links[n2][t]['end_points'][0], nod],
                                                  'points':links[n2][t]['points'][:nod_idx+1],
                                                  'nodes':links[n2][t]['nodes'][:nod_idx+1]}
                    links[n2][t] = {'end_points':[nod, links[n2][t]['end_points'][1]],
                                        'points':links[n2][t]['points'][nod_idx:],
                                        'nodes':links[n2][t]['nodes'][nod_idx:]}

                    break

In [172]:
for i in links:
    for j in links[i]:
         links[i][j]['end_points'] = [str(k) for k in links[i][j]['end_points']]
#         links[i][j]['nodes'] = [str(k) for k in links[i][j]['nodes']]
json.dump(links, open('./links_BL_L_with_synapses_attached.json','w'))
for i in neuron_synapse_nodes:
    neuron_synapse_nodes[i] = [str(j) for j in neuron_synapse_nodes[i]]
json.dump(neuron_synapse_nodes, open('./neurons_synapse_nodes_BL_L.json','w'))

# clean branches

In [237]:
links_clean = jlo('./links_BL_L_with_synapses_attached.json')

In [238]:
## delete neurons with no synapses ##
for i in neuron_synapse_nodes:
    if len(neuron_synapse_nodes[i]) == 0:
        del links_clean[i]

In [311]:
for idx in links_clean:
    nodes = array([links_clean[idx][j]['end_points'][0] for j in links_clean[idx]]+[links_clean[idx][j]['end_points'][1] for j in links_clean[idx]])
    n_list = array(list(set(nodes)))
    degrees = array([len(where(nodes==i)[0]) for i in n_list])
    ends = n_list[where(degrees==1)[0]]
    flag = 1
    while flag > 0:
        tmp = list(links_clean[idx].keys())
        flag = 0
        for i in tmp:
            if links_clean[idx][i]['end_points'][0] in ends or links_clean[idx][i]['end_points'][1] in ends:
                if len(set(links_clean[idx][i]['nodes']).intersection(set(neuron_synapse_nodes[idx])))==0:
                    del links_clean[idx][i]
                    flag = 1
                elif (links_clean[idx][i]['end_points'][0] in ends) and (links_clean[idx][i]['end_points'][1] in neuron_synapse_nodes[idx]):
                    del links_clean[idx][i]
                    flag = 1
                elif (links_clean[idx][i]['end_points'][1] in ends) and (links_clean[idx][i]['end_points'][0] in neuron_synapse_nodes[idx]):
                    del links_clean[idx][i]
                    flag = 1
        nodes = array([links_clean[idx][j]['end_points'][0] for j in links_clean[idx]]+[links_clean[idx][j]['end_points'][1] for j in links_clean[idx]])
        n_list = array(list(set(nodes)))
        degrees = array([len(where(nodes==i)[0]) for i in n_list])
        ends = n_list[where(degrees==1)[0]]              
        

In [312]:
json.dump(links_clean, open('./links_cleaned_branches_with_synapses_BL_L.json','w'))

In [323]:
## clear degree-2 nodes ##
for idx in links_clean:
    nodes = array([links_clean[idx][j]['end_points'][0] for j in links_clean[idx]]+[links_clean[idx][j]['end_points'][1] for j in links_clean[idx]])
    n_list = array(list(set(nodes)))
    degrees = array([len(where(nodes==i)[0]) for i in n_list])
#     ends = n_list[where(degrees==1)[0]]
    targets = n_list[where(degrees==2)[0]]
    while len(targets) > 0:
#         tmp = list(links_clean[idx].keys())
        for i in targets:
            pair = []
            for j in links_clean[idx]:
                if links_clean[idx][j]['end_points'][0]==i or links_clean[idx][j]['end_points'][1]==i:
                    pair.append(j)
            if len(pair) == 2:
                if links_clean[idx][pair[0]]['end_points'][0] == i:
                    if links_clean[idx][pair[1]]['end_points'][0] == i:
                        links_clean[idx][pair[0]]['end_points'] = [links_clean[idx][pair[1]]['end_points'][1],links_clean[idx][pair[0]]['end_points'][1]]
                        links_clean[idx][pair[0]]['points'] = links_clean[idx][pair[1]]['points'][::-1][:-1] + links_clean[idx][pair[0]]['points']
                        links_clean[idx][pair[0]]['nodes'] = links_clean[idx][pair[1]]['nodes'][::-1][:-1] + links_clean[idx][pair[0]]['nodes']
                        del links_clean[idx][pair[1]]
                    else:
                        links_clean[idx][pair[0]]['end_points'] = [links_clean[idx][pair[1]]['end_points'][0],links_clean[idx][pair[0]]['end_points'][1]]
                        links_clean[idx][pair[0]]['points'] = links_clean[idx][pair[1]]['points'][:-1] + links_clean[idx][pair[0]]['points']
                        links_clean[idx][pair[0]]['nodes'] = links_clean[idx][pair[1]]['nodes'][:-1] + links_clean[idx][pair[0]]['nodes']
                        del links_clean[idx][pair[1]]
                else:
                    if links_clean[idx][pair[1]]['end_points'][0] == i:
                        links_clean[idx][pair[0]]['end_points'] = [links_clean[idx][pair[0]]['end_points'][0],links_clean[idx][pair[1]]['end_points'][1]]
                        links_clean[idx][pair[0]]['points'] = links_clean[idx][pair[0]]['points'][:-1] + links_clean[idx][pair[1]]['points']
                        links_clean[idx][pair[0]]['nodes'] = links_clean[idx][pair[0]]['nodes'][:-1] + links_clean[idx][pair[1]]['nodes']
                        del links_clean[idx][pair[1]]
                    else:
                        links_clean[idx][pair[0]]['end_points'] = [links_clean[idx][pair[0]]['end_points'][0],links_clean[idx][pair[1]]['end_points'][0]]
                        links_clean[idx][pair[0]]['points'] = links_clean[idx][pair[0]]['points'][:-1] + links_clean[idx][pair[1]]['points'][::-1]
                        links_clean[idx][pair[0]]['nodes'] = links_clean[idx][pair[0]]['nodes'][:-1] + links_clean[idx][pair[1]]['nodes'][::-1]
                        del links_clean[idx][pair[1]]
            elif len(pair) == 1:
                del links_clean[idx][pair[0]]
        nodes = array([links_clean[idx][j]['end_points'][0] for j in links_clean[idx]]+[links_clean[idx][j]['end_points'][1] for j in links_clean[idx]])
        n_list = array(list(set(nodes)))
        degrees = array([len(where(nodes==i)[0]) for i in n_list])
#         ends = n_list[where(degrees==1)[0]]
        targets = n_list[where(degrees==2)[0]]              
   
                     
        

In [325]:
json.dump(links_clean, open('./links_cleaned_branches_with_synapses_no_degree_2_nodes_BL_L.json','w'))

In [326]:
links_clean = jlo('./links_cleaned_branches_with_synapses_BL_L.json')

In [ ]:
## build full network ##
links_f = {}
for idx in links:
    for i in links[idx]:
        links_f[str(len(links_f))] = {'end_points':[str(links[idx][i]['points'][0]), str(links[idx][i]['points'][-1])], 'points':links[idx][i]['points']}
json.dump(links_f, open('./links_of_full_network_BL_L.json','w'))        

In [ ]:
## delete twigs ##
nodes = array([links_f[j]['end_points'][0] for j in links_f]+[links_f[j]['end_points'][1] for j in links_f])
n_list = array(list(set(nodes)))
degrees = array([len(where(nodes==i)[0]) for i in n_list])
ends = n_list[where(degrees==1)[0]]
flag = 1
while flag > 0:
    tmp = list(links_f.keys())
    flag = 0
    for i in tmp:
        if links_f[i]['end_points'][0] in ends or links_f[i]['end_points'][1] in ends:
            del links_f[i]
            flag = 1
#             elif (links_f[i]['end_points'][0] in ends) and (links_f[i]['end_points'][1] in neuron_synapse_nodes[idx]):
#                 del links_f[i]
#                 flag = 1
#             elif (links_f[i]['end_points'][1] in ends) and (links_f[i]['end_points'][0] in neuron_synapse_nodes[idx]):
#                 del links_f[i]
#                 flag = 1
    nodes = array([links_f[j]['end_points'][0] for j in links_f]+[links_f[j]['end_points'][1] for j in links_f])
    n_list = array(list(set(nodes)))
    degrees = array([len(where(nodes==i)[0]) for i in n_list])
    ends = n_list[where(degrees==1)[0]]              
json.dump(links_f, open('./links_of_full_network_no_twigs_BL_L.json','w'))        

In [ ]:
## clear degree-2 nodes ##
nodes = array([links_f[j]['end_points'][0] for j in links_f]+[links_f[j]['end_points'][1] for j in links_f])
n_list = array(list(set(nodes)))
degrees = array([len(where(nodes==i)[0]) for i in n_list])
#     ends = n_list[where(degrees==1)[0]]
targets = n_list[where(degrees==2)[0]]
while len(targets) > 0:
#         tmp = list(links_clean[idx].keys())
    for i in targets:
        pair = []
        for j in links_f:
            if links_f[j]['end_points'][0]==i or links_f[j]['end_points'][1]==i:
                pair.append(j)
        if len(pair) == 2:
            if links_f[pair[0]]['end_points'][0] == i:
                if links_f[pair[1]]['end_points'][0] == i:
                    links_f[pair[0]]['end_points'] = [links_f[pair[1]]['end_points'][1],links_f[pair[0]]['end_points'][1]]
                    links_f[pair[0]]['points'] = links_f[pair[1]]['points'][::-1][:-1] + links_f[pair[0]]['points']
#                     links_f[pair[0]]['nodes'] = links_f[pair[1]]['nodes'][::-1][:-1] + links_f[pair[0]]['nodes']
                    del links_f[pair[1]]
                else:
                    links_f[pair[0]]['end_points'] = [links_f[pair[1]]['end_points'][0],links_f[pair[0]]['end_points'][1]]
                    links_f[pair[0]]['points'] = links_f[pair[1]]['points'][:-1] + links_f[pair[0]]['points']
#                     links_f[pair[0]]['nodes'] = links_f[pair[1]]['nodes'][:-1] + links_f[pair[0]]['nodes']
                    del links_f[pair[1]]
            else:
                if links_f[pair[1]]['end_points'][0] == i:
                    links_f[pair[0]]['end_points'] = [links_f[pair[0]]['end_points'][0],links_f[pair[1]]['end_points'][1]]
                    links_f[pair[0]]['points'] = links_f[pair[0]]['points'][:-1] + links_f[pair[1]]['points']
#                     links_f[pair[0]]['nodes'] = links_f[pair[0]]['nodes'][:-1] + links_f[pair[1]]['nodes']
                    del links_f[pair[1]]
                else:
                    links_f[pair[0]]['end_points'] = [links_f[pair[0]]['end_points'][0],links_f[pair[1]]['end_points'][0]]
                    links_f[pair[0]]['points'] = links_f[pair[0]]['points'][:-1] + links_f[pair[1]]['points'][::-1]
#                     links_f[pair[0]]['nodes'] = links_f[pair[0]]['nodes'][:-1] + links_f[pair[1]]['nodes'][::-1]
                    del links_f[pair[1]]
        elif len(pair) == 1 and len(links_f[pair[0]]['points']) <4:
            del links_f[pair[0]]
        elif len(pair)>2:
            print(pair)
    nodes = array([links_f[j]['end_points'][0] for j in links_f]+[links_f[j]['end_points'][1] for j in links_f])
    n_list = array(list(set(nodes)))
    degrees = array([len(where(nodes==i)[0]) for i in n_list])
#         ends = n_list[where(degrees==1)[0]]
    targets = n_list[where(degrees==2)[0]]              

json.dump(links_f, open('./links_of_full_network_no_twigs_no_degree_2_nodes_BL_L.json','w'))
        

In [ ]:
net = {'links':links_f}
edge_list = [net['links'][i]['end_points'] for i in net['links']]
adj_list = co.defaultdict(list)
for i,j in edge_list:
    adj_list[i].append(j)
    adj_list[j].append(i)

for i in adj_list:
    adj_list[i] = list(set(adj_list[i]))

In [ ]:
## measrue nGLN ##

network = gln.simplify_v8(adj_list,net,core=95)
network.find_base_loops(2)
network.get_min_loop_set()
network.separate_base_loops()
network.cal_normalization_const_parallel()
norm = network.new_norm
print(norm)
network.cal_m2_old_parallel()
network.cal_m2_normalized_new(norm = norm)
print(network.m2_normalized)